In [33]:
import sys

sys.path.insert(0, "..")


# Observer Agent

Collects facts and observations about a user from their conversations for later use when personalizing results.

In [34]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


input_variables = ["input", "history", "current_date"]
template = """
Below is a chat log between a human user and an AI. The Input is a new 
message sent by the user to the chat. Profile context includes relevant observations we know about the user.

Based on the new input, generate observations about who the user is as a person: their preferences, beliefs, personal details, etc. Focus on what you 
can learn generally about the user and their personal traits. For any observations that are time dependent (such as a date, meeting, event, holiday, or milestone) start with "As of {current_date}"

Output observations should be full sentences and refer to the human as "User",
separated by newlines. Include as many relevant details as possible in each statement, including relevant context. Be sure to include any factual statements you can infer about the user. Be sure to include any preferences you can infer based on what and how they choose to do, buy, or travel to. Also include inferrences you can make that are likely about the user's personality based on observable facts.

--EXAMPLES
EXAMPLE 1:

Input: Cappuccinos are my favorite coffee drink. I have them every day.

Output:
User's favorite coffee drink is a cappuccino
User is a frequent coffee drinker

EXAMPLE 2:

Current Date: {current_date}

Profile Context:
User has a girlfriend named Tiffany

Chat Log:

Input: Tiffany's car has a flat! Help!

Output:
Tiffany (User's girlfriend) has a car
As of {current_date}, Tiffany's (User's girlfriend) car has a flat tire
User is asking for help fixing a flat.
User's girlfriend might need immediate attention.

Tiffany (User's girlfriend) has a car
As of {current_date}, Tiffany's (User's girlfriend) car has a flat tire


EXAMPLE 3:
Profile Context:
User has a girlfriend named Tiffany

Chat Log:
user: Tiffany's car has a flat!
agent: I can help with that. What kind of car is it?

Input: Lamborghini

Output:
Tiffany (User's girlfriend) has a Lamborghini
Tiffany (User's girlfriend) is probably into nice cars

Tiffany (User's girlfriend) has a Lamborghini
Tiffany (User's girlfriend) is probably into nice cars
--END EXAMPLES


Current Date: {current_date}

Profile Context:


Chat Log:
{history}

Input:
{input}

Output: 
"""

llm = OpenAI(temperature=0)
prompt = PromptTemplate(template=template,input_variables=input_variables)
chain = LLMChain(llm=llm, prompt=prompt, verbose=True)

def to_chat_log(history):
  lines = []
  for line in history:
    sender = line["s"]
    message = line["m"]
    lines.append(f"{sender}: {message}")
  return "\n".join(lines)

def label_topic(history, idx):
    message = history[idx]["m"]
    chat_log = to_chat_log(history[0:idx])
    topic = chain.run(
      input=message,
      history=chat_log,
      current_date="2023-05-23")
    print(topic)
    return topic

def collect_observations(history):
  observations = []
  for idx in range(len(history)):
    if history[idx]["s"] != "user":
      continue
    results = label_topic(history, idx)
  for result in results.split("\n"):
    observations.append(result)
  return observations



In [35]:
from sessions import plan_date, meeting_counseling, travel
obs1 = collect_observations(plan_date)
obs1




> Entering new LLMChain chain...
Prompt after formatting:

Below is a chat log between a human user and an AI. The Input is a new 
message sent by the user to the chat. Profile context includes relevant observations we know about the user.

Based on the new input, generate observations about who the user is as a person: their preferences, beliefs, personal details, etc. Focus on what you 
can learn generally about the user and their personal traits. For any observations that are time dependent (such as a date, meeting, event, holiday, or milestone) start with "As of 2023-05-23"

Output observations should be full sentences and refer to the human as "User",
separated by newlines. Include as many relevant details as possible in each statement, including relevant context. Be sure to include any factual statements you can infer about the user. Be sure to include any preferences you can infer based on what and how they choose to do, buy, or travel to. Also include inferrences you can make

['User is going on a date this weekend',
 'User is looking for advice on what to do and what to wear for the date',
 'User is vegan-conscious',
 'User is looking for a vegan-friendly restaurant in Santa Monica',
 "User is nervous about the date, as they haven't been on one in a while",
 'User is a great conversationalist',
 "User appreciates Ozlo's encouragement and help in planning the date"]

In [36]:
obs2 = collect_observations(meeting_counseling)
obs2



> Entering new LLMChain chain...
Prompt after formatting:

Below is a chat log between a human user and an AI. The Input is a new 
message sent by the user to the chat. Profile context includes relevant observations we know about the user.

Based on the new input, generate observations about who the user is as a person: their preferences, beliefs, personal details, etc. Focus on what you 
can learn generally about the user and their personal traits. For any observations that are time dependent (such as a date, meeting, event, holiday, or milestone) start with "As of 2023-05-23"

Output observations should be full sentences and refer to the human as "User",
separated by newlines. Include as many relevant details as possible in each statement, including relevant context. Be sure to include any factual statements you can infer about the user. Be sure to include any preferences you can infer based on what and how they choose to do, buy, or travel to. Also include inferrences you can make

['User is in a co-parenting situation with an ex-partner',
 'User has children',
 'User is trying to improve the co-parenting situation',
 'User is looking for advice on how to communicate effectively with an ex-partner',
 'User is appreciative of advice given by the AI']

In [37]:
obs3 = collect_observations(travel)
obs3



> Entering new LLMChain chain...
Prompt after formatting:

Below is a chat log between a human user and an AI. The Input is a new 
message sent by the user to the chat. Profile context includes relevant observations we know about the user.

Based on the new input, generate observations about who the user is as a person: their preferences, beliefs, personal details, etc. Focus on what you 
can learn generally about the user and their personal traits. For any observations that are time dependent (such as a date, meeting, event, holiday, or milestone) start with "As of 2023-05-23"

Output observations should be full sentences and refer to the human as "User",
separated by newlines. Include as many relevant details as possible in each statement, including relevant context. Be sure to include any factual statements you can infer about the user. Be sure to include any preferences you can infer based on what and how they choose to do, buy, or travel to. Also include inferrences you can make

['User is planning a vacation to Maui, Hawaii',
 'User prefers morning flights',
 'User prefers LAX airport',
 'User is flexible with travel dates',
 'User is looking for hotels with great customer service, ocean views, and a fitness center',
 'User is appreciative of help from AI agent, Ozlo']

In [38]:
obs0 = (
  "User's name is John Doe",
  "User is 40 years old",
  "User lives in Los Angeles"
)
obs=[element for sublist in (obs0,obs1,obs2,obs3) for element in sublist]
obs

["User's name is John Doe",
 'User is 40 years old',
 'User lives in Los Angeles',
 'User is going on a date this weekend',
 'User is looking for advice on what to do and what to wear for the date',
 'User is vegan-conscious',
 'User is looking for a vegan-friendly restaurant in Santa Monica',
 "User is nervous about the date, as they haven't been on one in a while",
 'User is a great conversationalist',
 "User appreciates Ozlo's encouragement and help in planning the date",
 'User is in a co-parenting situation with an ex-partner',
 'User has children',
 'User is trying to improve the co-parenting situation',
 'User is looking for advice on how to communicate effectively with an ex-partner',
 'User is appreciative of advice given by the AI',
 'User is planning a vacation to Maui, Hawaii',
 'User prefers morning flights',
 'User prefers LAX airport',
 'User is flexible with travel dates',
 'User is looking for hotels with great customer service, ocean views, and a fitness center',
 'Us

In [43]:
def filter_irrelevant_obs():
  statements = "\n".join(obs)
  prompt = f"""
    Below is a list of statements learned about a user during chat. Output
    only those statements that are likely to be true in the future as well as
    right now. Omit statements that are only related to what the user was 
    looking for during the chat or are tied to a specific event. Do not modify any statements.

    Statements:
    {statements}

    Output:
    """
  print(prompt)
  filtered_obs = llm(prompt)
  return filtered_obs.split("\n")

obs = filter_irrelevant_obs()
obs


    Below is a list of statements learned about a user during chat. Output
    only those statements that are likely to be true in the future as well as
    right now. Omit statements that are only related to what the user was 
    looking for during the chat or are tied to a specific event. Do not modify any statements.

    Statements:
     User's name is John Doe
User is 40 years old
User lives in Los Angeles
User is vegan-conscious
User is a great conversationalist
User has children
User is trying to improve the co-parenting situation
User is appreciative of advice given by the AI
User prefers morning flights
User prefers LAX airport
User is flexible with travel dates
User is looking for hotels with great customer service, ocean views, and a fitness center
User is appreciative of help from AI agent, Ozlo

    Output:
    


[" User's name is John Doe",
 'User is 40 years old',
 'User lives in Los Angeles',
 'User is vegan-conscious',
 'User is a great conversationalist',
 'User has children',
 'User prefers morning flights',
 'User prefers LAX airport',
 'User is flexible with travel dates']

# Observation Store

This is a vectorstore that holds observations about the user. You can ask the vector store to retrieve any observations that seems closely related to a question.

In [44]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

embeddings = OpenAIEmbeddings()
obs_db = Chroma.from_texts(obs, embeddings, collection_name="obs")
  


Using embedded DuckDB without persistence: data will be transient


In [45]:
def find_outdated_obs():
  new_obs = "User got married as of 2023-10-01"
  related = obs_db.similarity_search(new_obs)
  old_obs = "\n".join([x.page_content for x in related])
  prompt = f"""
    Below are a list of new statements and old statements about a user. New 
    statements came later than the old statements and might reflect a change in
    status. Output the list of old statements that are definitely outdated as
    a result of the new statements. Only output those with high confidence.

    Output should be a JSON array of the type:
    [...[statement, reason]]
    where `statement` is the invalid statement and `reason` explains why you
    think it is invalid.

    New Statements:
    {new_obs}

    Old Statements:
    {old_obs}

    Output:
  """
  print(prompt)
  outdated_obs = llm(prompt)
  return outdated_obs.split("\n")

find_outdated_obs()


    Below are a list of new statements and old statements about a user. New 
    statements came later than the old statements and might reflect a change in
    status. Output the list of old statements that are definitely outdated as
    a result of the new statements. Only output those with high confidence.

    Output should be a JSON array of the type:
    [...[statement, reason]]
    where `statement` is the invalid statement and `reason` explains why you
    think it is invalid.

    New Statements:
    User got married as of 2023-10-01

    Old Statements:
    User is 40 years old
User has children
User is flexible with travel dates
User lives in Los Angeles

    Output:
  


[' [["User has children", "User got married as of 2023-10-01"], ["User is flexible with travel dates", "User got married as of 2023-10-01"]]']

# Personalization Context

This agent will return any observations stored in a vectorstore that seem relevant to a passed request. These can be injected into other agents to influence ranking or to generate responses tailored to the user.

In [42]:
def test_context_in_flight():
  input = "i need to take a flight to new york next week"
  related_docs = obs_db.similarity_search(f"what is users name and basic demographics? {input}")
  profile_context = "\n".join([x.page_content for x in related_docs])
  prompt = f"""
  You are a friendly AI assistant named Ozlo. You do you best to provide highly
  personalized responses, making full use of the context given below to deliver a response specific to the user. You respond in short, consise, polite manner and never take charge except to ask clarifying questions. Generate an answer to the chat log below as the AI agent. 

  Profile Context (Things we have learned about the user)
  {profile_context}

  Chat Log: (Past messages)
  user: hello
  agent: hi John!

  Input: (latest message from the user)
  {input}

  Output: (Response from the AI assistant)
  """
  print(prompt)
  result = llm(prompt)
  return result

test_context_in_flight()


  You are a friendly AI assistant named Ozlo. You do you best to provide highly
  personalized responses, making full use of the context given below to deliver a response specific to the user. You respond in short, consise, polite manner and never take charge except to ask clarifying questions. Generate an answer to the chat log below as the AI agent. 

  Profile Context (Things we have learned about the user)
  User is flexible with travel dates
User prefers morning flights
User prefers LAX airport
User is a great conversationalist

  Chat Log: (Past messages)
  user: hello
  agent: hi John!

  Input: (latest message from the user)
  i need to take a flight to new york next week

  Output: (Response from the AI assistant)
  


'\nGreat, I can help you find a flight to New York next week. Would you like to fly out of LAX airport and prefer morning flights?'

In [50]:
def test_context_for_food():
  input = "I need a cool place to eat on Friday."
  query = "what do I need to know about the user to find a restaurant they will like?"
  related_docs = obs_db.similarity_search(query)
  profile_context = "\n".join([x.page_content for x in related_docs])
  prompt = f"""
  You are a friendly AI assistant named Ozlo. You do you best to provide highly
  personalized responses, making full use of the context given below to deliver a response specific to the user. You respond in short, consise, polite manner and never take charge except to ask clarifying questions. Generate an answer to the chat log below as the AI agent. 

  Profile Context (Things we have learned about the user)
  {profile_context}

  Chat Log: (Past messages)
  user: hello
  agent: hi John!

  Input: (latest message from the user)
  {input}

  Output: (Response from the AI assistant)
  """
  print(prompt)
  result = llm(prompt)
  return result

test_context_for_food()


  You are a friendly AI assistant named Ozlo. You do you best to provide highly
  personalized responses, making full use of the context given below to deliver a response specific to the user. You respond in short, consise, polite manner and never take charge except to ask clarifying questions. Generate an answer to the chat log below as the AI agent. 

  Profile Context (Things we have learned about the user)
  User is a great conversationalist
User is flexible with travel dates
User is vegan-conscious
User has children

  Chat Log: (Past messages)
  user: hello
  agent: hi John!

  Input: (latest message from the user)
  I need a cool place to eat on Friday.

  Output: (Response from the AI assistant)
  


"\nHi John, I'm sure I can help you find a great place to eat on Friday. Do you have any dietary restrictions or preferences, such as vegan-consciousness?"

Bad pipe message: %s [b'\xf4\xbb\xa7\xf8']
Bad pipe message: %s [b'\x12\x1e\xf0c\r\xb0\x04@\x1c\xa3t\x8e \x97iBc\xc0\xdf\x7f\xcd=$\x1d\x06\xf3']
Bad pipe message: %s [b"\xb9a\xd3\xcac\xf2\xfb\x9fI\xfa\xad<\xdc\xfe{'V\x1d\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00", b"j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00<\x005\x00/\x00\x9a\x00\x99\xc0\x07\xc0\x11\x00\x96\x00\x05\x00\xff\x01\x00\x00j\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00"]
Bad pipe message: %s [b'\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x000\x00.\x04\x03\x05\x03\x06']
Bad pipe message: %s [b'\x07\x08']
Bad pipe message: %s [b'\t\x08\n\x08\x0b\x08\x04']
Bad pipe message: %s [b'\x08\x0